In [12]:
import pandas as pd
import json
import warnings
import os
import random

warnings.filterwarnings(action='ignore')

#### Lendo as atividades em dados brutos de cada arquivo e passando para um Dataframe

In [13]:
# Caminho da pasta que terá iteração nos seus arquivos
folder_path = '.../running_inactivity_prediction/athletes_activities/athlete4'

# Leitura de cada uma das atividades no formato json 
raw_activities_data = [json.load(open(os.path.join(folder_path, filename),"rb")) for filename in os.listdir(folder_path)]

df_raw_activities_data = pd.DataFrame(raw_activities_data)
df_raw_activities_data.head() 

,id,type,app_id,start_epoch_ms,end_epoch_ms,last_modified,active_duration_ms,session,delete_indicator,summaries,sources,tags,change_tokens,metric_types,metrics,moments
0,0a611c05-cfad-44e5-bb07-e7c53dc61534,run,com.nike.sport.running.ios,1680775468014,1680777275965,1680779398427,1803188,False,False,"[{'metric': 'distance', 'summary': 'total', 's...","[com.nike.running.ios.calculatedcalories, com....",{'com.nike.name': 'quinta-feira corrida matuti...,[a24fcd6ca956e751162ae236d7e08a621be74a66bdd35...,"[elevation, distance, latitude, pace, calories...","[{'type': 'distance', 'unit': 'KM', 'source': ...","[{'key': 'split_km', 'value': '1', 'timestamp'..."
1,2f5afc63-13d3-4eb4-b802-fda070c932f9,run,com.nike.sport.running.ios,1598611298790,1598613238109,1603923963098,1924517,False,False,"[{'metric': 'distance', 'summary': 'total', 's...","[com.nike.pacecalculator.v1, com.nike.running....","{'com.nike.name': '28/08/2020', 'com.nike.runn...",[6ef8001f17acf34c39813ea942494a2a043235d961b01...,"[elevation, distance, latitude, pace, calories...","[{'type': 'distance', 'unit': 'KM', 'source': ...","[{'key': 'split_km', 'value': '1', 'timestamp'..."
2,c0a3d98e-20d7-498e-a756-357efd232627,run,com.nike.sport.running.ios,1639648390026,1639650278655,1639650405337,1863930,False,False,"[{'metric': 'calories', 'summary': 'total', 's...","[com.nike.running.ios.calculatedcalories, com....",{'com.nike.name': 'quinta-feira corrida matuti...,[58861a75c5aa1e05956d805abdde9e5b17d3be08c01ef...,"[elevation, distance, latitude, pace, calories...","[{'type': 'distance', 'unit': 'KM', 'source': ...","[{'key': 'split_km', 'value': '1', 'timestamp'..."
3,c1ae11a1-03bf-4eb5-9d68-648f5aba5cb8,run,com.nike.sport.running.ios,1607258917177,1607260657221,1615165514363,1722992,False,False,"[{'metric': 'steps', 'summary': 'total', 'sour...","[com.nike.pacecalculator.v1, com.nike.running....","{'com.nike.name': '06/12/2020', 'com.nike.runn...",[6873ef008f3a8047d424c30acbea75e6884689d9aa5ed...,"[elevation, distance, latitude, pace, calories...","[{'type': 'distance', 'unit': 'KM', 'source': ...","[{'key': 'split_km', 'value': '1', 'timestamp'..."
4,c4f82745-9336-4176-89ae-56034a8479a7,run,com.nike.sport.running.ios,1662977694328,1662979488990,1662980276877,1787573,False,False,"[{'metric': 'ascent', 'summary': 'total', 'sou...","[com.nike.running.ios.calculatedcalories, com....",{'com.nike.name': 'segunda-feira corrida matut...,[0a8d1ffb812fdbb1660f1537545328ed73215c632ba81...,"[elevation, distance, latitude, pace, calories...","[{'type': 'distance', 'unit': 'KM', 'source': ...","[{'key': 'split_km', 'value': '1', 'timestamp'..."


#### Sumário dos dados de corrida

In [14]:
# Atividade aleatória
random_activity = random.randint(0, len(df_raw_activities_data))

pd.DataFrame(df_raw_activities_data['summaries'][random_activity])

,metric,summary,source,app_id,value
0,distance,total,com.nike.running.ios.coremotion,com.nike.sport.running.ios,6.020947
1,speed,mean,com.nike.running.ios.coremotion,com.nike.sport.running.ios,9.622627
2,nikefuel,total,com.nike.nikefuelengine.1.0-A5.0,com.nike.sport.running.ios,1562.668662
3,pace,mean,com.nike.running.ios.coremotion,com.nike.sport.running.ios,6.235304
4,steps,total,com.nike.running.ios.coremotion,com.nike.sport.running.ios,6176.000000
5,nikefuel,mean,com.nike.nikefuelengine.1.0-A5.0,com.nike.sport.running.ios,41.624064
6,descent,total,com.nike.running.ios.coremotion,com.nike.sport.running.ios,28.864693
7,calories,total,com.nike.running.ios.calculatedcalories,com.nike.sport.running.ios,336.410000
8,ascent,total,com.nike.running.ios.coremotion,com.nike.sport.running.ios,26.864458


#### Extraindo a distância, pace e velocidade dos dados brutos e passando para um dicionário

In [15]:
# Lista das metricas que serão identificadas
metrics = ['distance', 'pace', 'speed']

# Dicionário com as features que serão criadas com o valor de cada atividade
df_athlete_dict = {
    'total_distance (km)': [], 
    'pace (min/km)': [],
    'velocity (km/h)': []    
}

# Iteração no número de atividades
for activity in range(len(df_raw_activities_data)):
    # Iteração para encontrar a linha com a métrica desejada e adicionar seu valor no dicionário com a chave certa
    for values in df_raw_activities_data['summaries'][activity]: # por atividade
        if (values['metric'] == metrics[0]): 
            df_athlete_dict['total_distance (km)'].append(round(values['value'],2))
        elif (values['metric'] == metrics[1]):
            df_athlete_dict['pace (min/km)'].append(round(values['value'],2))
        elif (values['metric'] == metrics[2]):
            df_athlete_dict['velocity (km/h)'].append(round(values['value'],2))

#### Criando um Dataframe novo a partir do dicionário

In [16]:
# Criação do DataFrame de atividades
df_athlete_activities = pd.DataFrame(df_athlete_dict)

#### Extraindo a duração das atividades e passando para o Dataframe novo

In [17]:
# Tempo inicial da atividade
start = pd.to_datetime(df_raw_activities_data['start_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('s')

# Tempo final da atividade
end = pd.to_datetime(df_raw_activities_data['end_epoch_ms'],unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.round('s')

# Criação da coluna com tempo total em minutos a partir da subtração dos tempos final e inicial
df_athlete_activities['total_time (min)'] = (end - start).apply(lambda x: x.total_seconds() // 60)

#### Extraindo a data das atividades e passando para o Dataframe novo

In [18]:
# Criando a coluna para a data da atividade
activity_dates = pd.to_datetime(df_raw_activities_data['start_epoch_ms'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo').dt.tz_localize(None).dt.normalize()
df_athlete_activities.insert(0, 'activity_date', activity_dates)

# Ordenando o dataframe pela data menor à maior
df_athlete_activities.sort_values('activity_date', ascending=True, inplace=True)

#### Extraindo o tipo das atividades e passando para o Dataframe novo

In [19]:
# Criando coluna com o tipo da atividade
df_athlete_activities['activity_type'] = df_raw_activities_data['type']

In [20]:
# Resetando o index
df_athlete_activities.reset_index(drop=True, inplace=True)
df_athlete_activities

,activity_date,total_distance (km),pace (min/km),velocity (km/h),total_time (min),activity_type
0,2020-06-13,8.06,7.07,8.49,58.0,run
1,2020-06-16,8.04,6.69,8.96,53.0,run
2,2020-06-20,10.05,6.51,9.21,65.0,run
3,2020-06-25,5.03,6.34,9.46,31.0,run
4,2020-06-28,7.03,6.32,9.49,44.0,run
...,...,...,...,...,...,...
145,2023-09-12,5.02,6.16,9.73,31.0,run
146,2023-09-17,5.01,6.26,9.59,31.0,run
147,2023-09-19,5.02,6.06,9.90,30.0,run
148,2023-09-21,5.01,7.14,8.40,36.0,run
